In [1]:
!pip install pandas sentence-transformers scikit-learn

In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [4]:
df1=pd.read_csv("companyName.csv")

In [5]:
df1.head()

,CompanyName
0,Allcargo Logistics
1,Blue Dart Express
2,Gati Ltd
3,Transport Corporation of India
4,VRL Logistics


In [6]:
df2=pd.read_csv("merged_output_Personality_jan9.csv")

In [7]:
df2.head()

,Search Query,Image URL
0,Ratan Tata,https://encrypted-tbn0.gstatic.com/images?q=tb...
1,Dhirubhai Ambani,https://encrypted-tbn0.gstatic.com/images?q=tb...
2,Fouaad Mirza,https://encrypted-tbn0.gstatic.com/images?q=tb...
3,G. V. Ramanjaneyulu,https://encrypted-tbn0.gstatic.com/images?q=tb...
4,Dinesh K Sarraf,https://encrypted-tbn0.gstatic.com/images?q=tb...


In [8]:
df3=pd.read_csv("merged_output_logo_jan9.csv")

In [9]:
df3.head()

,Search Query,Image URL
0,SIFS India,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,Hindustan Unilever Limited,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
2,Okaya Power Group,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
3,Hindustan Zinc Limited,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
4,Veolia India,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."


In [ ]:
# Load the data
small_list_df = pd.read_csv("companyName.csv", header=None, names=["CompanyName"])
large_list_df = pd.read_csv("merged_output_logo_jan9.csv", header=None, names=["Search Query"])
small_careers = small_list_df["CompanyName"].tolist()
large_careers = large_list_df["Search Query"].tolist()
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode the career names into vectors
small_embeddings = model.encode(small_careers, convert_to_tensor=True)
large_embeddings = model.encode(large_careers, convert_to_tensor=True)

In [13]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Load the data
df1 = pd.read_csv("companyName.csv", header=None, names=["CompanyName"])
df3 = pd.read_csv("merged_output_logo_jan9.csv", header=None, names=["Search Query", "Image URL"])

# Convert columns to lists
small_careers = df1["CompanyName"].tolist()
large_careers = df3["Search Query"].tolist()
image_urls = df3["Image URL"].tolist()

# Initialize lists for results
exact_matches = []
semantic_matches = []

# Find exact matches first
for small_career in small_careers:
    exact_match = [
        (large_career, image_urls[idx]) 
        for idx, large_career in enumerate(large_careers) 
        if large_career.lower() == small_career.lower()
    ]
    if exact_match:
        exact_matches.append([small_career, exact_match[0][0], exact_match[0][1]])

# Save the exact matches to a DataFrame
exact_matches_df = pd.DataFrame(exact_matches, columns=["CompanyName", "Matched Search Query", "Image URL"])
exact_matches_df.to_csv("new_exact_matches_with_images.csv", index=False)

# Load the pre-trained Sentence-BERT model for semantic matching
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode the career names into vectors
small_embeddings = model.encode(small_careers, convert_to_tensor=True)
large_embeddings = model.encode(large_careers, convert_to_tensor=True)

# For semantic matches
for i, small_career in enumerate(small_careers):
    # Skip if it's already in the exact matches
    if any(small_career.lower() == exact_match[0].lower() for exact_match in exact_matches):
        continue
    
    # Compute cosine similarity between this career and all careers in the large list
    similarities = util.pytorch_cos_sim(small_embeddings[i], large_embeddings)
    
    # Get the index of the best match
    best_match_idx = similarities.argmax().item()
    
    # Get the corresponding Search Query and Image URL
    best_match_query = large_careers[best_match_idx]
    best_match_image_url = image_urls[best_match_idx]
    
    # Append the result to semantic matches
    semantic_matches.append([small_career, best_match_query, best_match_image_url])

# Convert the semantic matches to a DataFrame
semantic_matches_df = pd.DataFrame(semantic_matches, columns=["CompanyName", "Matched Search Query", "Image URL"])
semantic_matches_df.to_csv("new_matches_with_images.csv", index=False)

print("Matching complete!")
print("Exact matches saved to 'exact_matches_with_images.csv'")
print("Semantic matches saved to 'semantic_matches_with_images.csv'")


Matching complete!
Exact matches saved to 'exact_matches_with_images.csv'
Semantic matches saved to 'semantic_matches_with_images.csv'


In [14]:
df5=pd.read_csv("new_matches_with_images.csv")

In [15]:
df5.head()

,CompanyName,Matched Search Query,Image URL
0,CompanyName,The Craft Company,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,Allcargo Logistics,Allcargo Logistics,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
2,Blue Dart Express,Blue Dart Express,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
3,Gati Ltd,Gati Ltd,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
4,Transport Corporation of India,Transport Corporation of India,"data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."


In [16]:
import pandas as pd
import re

# Load the data
df5 = pd.read_csv("new_matches_with_images.csv")

# Function to normalize strings by removing spaces, commas, dots, special signs, and making it lowercase
def normalize_string(s):
    return re.sub(r'[^a-z0-9]', '', s.lower())

# Filter rows where normalized CompanyName matches normalized Matched Search Query
filtered_df = df5[
    df5.apply(lambda row: normalize_string(row["CompanyName"]) == normalize_string(row["Matched Search Query"]), axis=1)
]

# Save the filtered DataFrame to a new CSV
filtered_df.to_csv("new_v1_exact_normalized_matches.csv", index=False)

print("Filtered matches saved to 'exact_normalized_matches.csv'")


Filtered matches saved to 'exact_normalized_matches.csv'


In [ ]:
results = []

# For each career in the small list, find the most semantically similar careers in the large list
for i, small_career in enumerate(small_careers):
    # Compute cosine similarity between this career and all careers in the large list
    similarities = util.pytorch_cos_sim(small_embeddings[i], large_embeddings)
    
    # Get the indices of top matches, let's say top 5 matches
    top_matches = np.argpartition(similarities[0], -1)[-1:]
    # top_match_idx = similarities[0].argmax().item()  # Get the index of the most similar career
    # top_match = large_careers[top_match_idx]

    
    # Get the corresponding career names from large list
    matched_careers = [large_careers[idx] for idx in top_matches]
    
    # Save the career and matched careers to the results list
    results.append([small_career, ", ".join(matched_careers)])

# Convert the results to a DataFrame
output_df = pd.DataFrame(results, columns=["Career (Small List)", "Matched Careers (Large List)","Image URL"])

# Save the result to a CSV file
output_df.to_csv("matched_careers_top1.csv", index=False)

print("Matching complete! Results saved to matched_careers.csv")

In [ ]:
exact_matches = []
semantic_matches = []

# Find exact matches first
for small_career in small_careers:
    exact_match = [large_career for large_career in large_careers if large_career.lower() == small_career.lower()]
    if exact_match:
        exact_matches.append([small_career, ", ".join(exact_match)])

# Save the exact matches to a CSV file
exact_matches_df = pd.DataFrame(exact_matches, columns=["Career (Small List)", "Exact Matched Careers"])
exact_matches_df.to_csv("exact_matches_2.csv", index=False)

# Load the pre-trained Sentence-BERT model for semantic matching
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode the career names into vectors
small_embeddings = model.encode(small_careers, convert_to_tensor=True)
large_embeddings = model.encode(large_careers, convert_to_tensor=True)

# For each career in the small list, find the most semantically similar careers in the large list
for i, small_career in enumerate(small_careers):
    # Skip if it's already in the exact matches
    if any(small_career.lower() == exact_career[0].lower() for exact_career in exact_matches):
        continue
    
    # Compute cosine similarity between this career and all careers in the large list
    similarities = util.pytorch_cos_sim(small_embeddings[i], large_embeddings)
    
    # Get the indices of top matches, let's say top 5 matches
    top_matches = np.argpartition(similarities[0], -5)[-5:]
    
    # Get the corresponding career names from large list
    matched_careers = [large_careers[idx] for idx in top_matches]
    
    # Save the career and matched careers to the semantic matches list
    semantic_matches.append([small_career, ", ".join(matched_careers)])

# Convert the semantic matches to a DataFrame
semantic_matches_df = pd.DataFrame(semantic_matches, columns=["Career (Small List)", "Semantically Matched Careers"])

# Save the semantic matches result to a CSV file
semantic_matches_df.to_csv("semantic_matches_2.csv", index=False)

print("Matching complete! Exact matches saved to 'exact_matches.csv' and semantic matches saved to 'semantic_matches.csv'")